<a href="https://colab.research.google.com/github/fadiahanifa/Glaucoma-UNet/blob/main/Glaucoma%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Glaucoma Detection based on quantification of optic disc and optic cup**
Glaucoma is eye disease which is caused by increase of intraocular pressure. The pressure is damaging optic nerve head and could lead to partially or even entirely loss of eyesight if there is no appropriate treatment. In Indonesia, only 51.4% of glaucoma cases were only examined after they were at the severe stage of glaucoma namely when there is already significant damage to the eye or even when vision has been greatly reduced. Therefore, early glaucoma detection is crucial so that glaucoma can be treated as fast as possible. Currently, there are many developments in this field that have been carried out using various types of approaches. One of the approaches is by quantification of optic disc and cup’s such as cup to disc ratio. In this approach, we carried out the classification using a four step algorithm.




# Prerequisites

In [ ]:
!pip install --upgrade --no-cache-dir gdown
!gdown 1MI5jnTSwtJcmaUUa_0qekktw0w-LHuFA

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/uc?id=1MI5jnTSwtJcmaUUa_0qekktw0w-LHuFA
To: /content/Glaucoma Detection.zip
100% 151M/151M [00:08<00:00, 17.8MB/s]


In [ ]:
!unzip -u "/content/Glaucoma Detection.zip"

Archive:  /content/Glaucoma Detection.zip
   creating: Glaucoma Detection/
  inflating: Glaucoma Detection/.DS_Store  
  inflating: __MACOSX/Glaucoma Detection/._.DS_Store  
   creating: Glaucoma Detection/Sample Images/
   creating: Glaucoma Detection/Models and Templates/
  inflating: __MACOSX/Glaucoma Detection/._Models and Templates  
  inflating: Glaucoma Detection/Sample Images/drishtiGS_092.png  
  inflating: __MACOSX/Glaucoma Detection/Sample Images/._drishtiGS_092.png  
  inflating: Glaucoma Detection/Sample Images/n0317.jpg  
  inflating: __MACOSX/Glaucoma Detection/Sample Images/._n0317.jpg  
  inflating: Glaucoma Detection/Sample Images/drishtiGS_010.png  
  inflating: __MACOSX/Glaucoma Detection/Sample Images/._drishtiGS_010.png  
  inflating: Glaucoma Detection/Sample Images/n0294.jpg  
  inflating: __MACOSX/Glaucoma Detection/Sample Images/._n0294.jpg  
   creating: Glaucoma Detection/Models and Templates/model OD final/
  inflating: __MACOSX/Glaucoma Detection/Models an

In [ ]:
# Import libraries
import cv2
import math 
import numpy as np
import pickle
import tensorflow as tf

from google.colab import files
from keras import backend as K
from numpy.lib.twodim_base import diag
from skimage.transform import resize
from skimage.segmentation import slic

## Import Models

In [ ]:
# Localization Templates
image_R = cv2.imread('/content/Glaucoma Detection/Models and Templates/ROItemplateRed.png', 0)
image_G = cv2.imread('/content/Glaucoma Detection/Models and Templates/ROItemplateGreen.png', 0)
image_B = cv2.imread('/content/Glaucoma Detection/Models and Templates/ROItemplateBlue.png', 0)
image_templates = [image_R, image_G, image_B]

In [ ]:
# Evaluation metrics
def recall_m(y_true, y_pred):
    '''
    calculate the recall

    Parameters
    ----------
    y_true : array of int/float
      Ground truth (correct) target values.
    y_pred : array of int/float
      estimated value returned by a classifier.

    Returns
    ----------
    recall : float
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    '''
    calculate the precision

    Parameters
    ----------
    y_true : array of int/float
      Ground truth (correct) target values.
    y_pred : array of int/float
      estimated value returned by a classifier.

    Returns
    ----------
    recall : float
    '''
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def fscore(y_true, y_pred):
    '''
    calculate the fscore

    Parameters
    ----------
    y_true : array of int/float
      Ground truth (correct) target values.
    y_pred : array of int/float
      estimated value returned by a classifier.

    Returns
    ----------
    fscore : float
    '''
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# Segmentation Models
model_OD = tf.keras.models.load_model('/content/Glaucoma Detection/Models and Templates/model OD final', custom_objects={'fscore':fscore})
model_OC = tf.keras.models.load_model('/content/Glaucoma Detection/Models and Templates/model OC final', custom_objects={'fscore':fscore})

In [ ]:
# Classification Model
model2a_axes = pickle.load(open('/content/Glaucoma Detection/Models and Templates/model2a_axes.sav', 'rb'))

## Functions

### Localization Functions

In [ ]:
def extract_BR_map(src, mask, numSegments=50, sigma=10):
  '''
    This is the function of brightness map of retinal images extraction. 
    Brightness map is an image that contain well-segmented areas of retinal images
    that filled with the brightness of those areas. This maps give the brightness
    information of all area in retinal image. Since the Optic Disc typically
    the one of brightest area in retinal images, this map helps identify the
    location of Optic Disc.

    Parameters
    ----------
    src: array of int
      retinal images (RGB)
    mask: array of int
      mask for Masking out non-retinal area of the image
    numSegments: int
      The number of segment in the brightness map
    sigma: int
      the size of gaussian filter for blurring the image

    Returns
    ----------
    combined_map: array of int
      the combined brightness across R, G, B channels
  '''    

  #Resize imagen source
  H, W = src.shape[:2]
  
  # Resizing the image to quarter size for accelerating the computation
  resized_src = resize(src, output_shape=(H//4, W//4), mode = 'constant')
  resized_mask =  resize(mask, output_shape=(H//4, W//4), mode = 'constant')

  # Blurring the image tim improve the segment accuracy
  blur = cv2.GaussianBlur(resized_src, (37, 37), 0)

  # Segmenting the retinal image using SLIC algorithm 
  segments = slic(blur, n_segments=numSegments, sigma=sigma, mask=resized_mask)
  # boundary = mark_boundaries(resized_src, segments)

  labels = np.unique(segments)
  h, w = resized_src.shape[:2]

  # initiate the brightness map of R, G, and B channel
  g_map = np.zeros([h, w], np.float32)
  r_map = np.zeros([h, w], np.float32)
  b_map = np.zeros([h, w], np.float32)


  r_src = resized_src[:, :, 0]
  g_src = resized_src[:, :, 1]
  b_src = resized_src[:, :, 2]

  # Filling the segment with brightness of it
  for label in labels:
    # Brightness is define as the maximum intensity of each segment
    r_map[segments==label] = np.max(r_src[segments==label])
    g_map[segments==label] = np.max(g_src[segments==label])
    b_map[segments==label] = np.max(b_src[segments==label])

  # Resize back the brightness map to its original size
  r_resized_map = resize(r_map, output_shape=(H, W), mode='constant', preserve_range=True)
  g_resized_map = resize(g_map, output_shape=(H, W), mode='constant', preserve_range=True)
  b_resized_map = resize(b_map, output_shape=(H, W), mode='constant', preserve_range=True)

  # combining brightness map using average method
  combined_map = (r_resized_map + g_resized_map + b_resized_map)/3
  
  return combined_map

In [ ]:
def padding(NCC, h, w):
  '''
    This function will zero pad the NCC map to compensate the size difference
    compared to the retinal image

    Parameters
    ----------
    NCC: array of int
      Normalized Cross-Correlation map
    h: int
      height of source image
    w: int
      width of source image

    Returns
    ----------
    zeropadded_ncc: array of int
      zero padded NCC to the size of h and w
  '''   
  ver = (h - NCC.shape[0])/2
  hor = (w - NCC.shape[1])/2
  top = math.floor(ver)
  bottom = math.ceil(ver)
  left =  math.floor(hor)
  right = math.ceil(hor)
  value = 0
  borderType = cv2.BORDER_CONSTANT
  zeropadded_NCC = cv2.copyMakeBorder(NCC, top, bottom, left, right, borderType, value)
  return zeropadded_NCC

In [ ]:
def image_template_matching(claheRetinalImg, template, mask):
  '''
    This is a function for OD template matching. This will yields Normalized
    Correlation Coefficient (NCC) of R, G, and B channel. 

    Parameters
    ----------
    claheRetinalImg: array of int
      RGB retinal image that had been CLAHE-ed
    template: array of int
      OD template image
    mask: array of int
      Mask for masking out the non-retinal area

    Returns
    ----------
    NCC_maps: array of int
      NCC image of R, G, and B channel
  '''

  h, w = claheRetinalImg[0].shape[:2]
  NCC_maps = []
  for i, clahe_image in enumerate(claheRetinalImg):
    # checking clahe image isn't zeros 
    if len(np.unique(clahe_image)) > 0: 
      NCC = cv2.matchTemplate(clahe_image, template[i], cv2.TM_CCOEFF_NORMED)
      NCC = NCC + abs(np.min(NCC))
      NCC = padding(NCC, h, w)
      NCC = NCC * mask

    NCC_maps.append(NCC)
  
  return NCC_maps

In [ ]:
def OD_Localization(src, template_images, used_channels='rgb',
                  bright_on=True, test_on=False, 
                  r_coeff=1, g_coeff=1, b_coeff=1, br_coeff=1):
  '''
    Localizes OD by looking for the estimated disc center
    
    Parameters
    ----------
    src: array of int
      retinal images (RGB)
    template_images: array of int
      Optic Disc template images of R, G, B channles
    used_channels: string
      defining the used NCC channels for localization
      must contain r, g, or b
    NCC_on: boolean
      Use NCC map for localization
    bright_on: boolean
      Use brightness map for localization
    r_coeff: float
      coefficient of red_NCC map
      value between 0 to 1
    g_coeff: float
      coefficient of green_NCC map 
      value between 0 to 1
    b_coeff: float
      coefficient of blue_NCC map 
      value between 0 to 1
    br_coeff: float
      coefficient of brightness map 
      value between 0 to 1

    Return
    ----------
    disk_center: tuple of int (x, y)
      Optic Disc Center
    all_maps: array of int
      list of created maps for localization, returned only if test_on is True.
  '''

    
  h, w = src.shape[:2]

  # Implement CLAHE to the input image 
  clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))
  cl_img = clahe.apply(src[:, :, 1])
  
  zeros = np.zeros([h, w], np.uint8)
  img = [zeros, zeros, zeros]
  
  # Apply CLAHE to each of image channel 
  for channel in used_channels:
    if channel == 'r':
      img[0] = clahe.apply(src[:, :, 0])
    elif channel == 'g':
      img[1] = clahe.apply(src[:, :, 1])
    elif channel == 'b':
      image = clahe.apply(src[:, :, 2])
      img[2] = image
    
  # Mask Out outside retinal image
  blurred = cv2.GaussianBlur(cl_img, (7, 7), 0)
  (T, mask) = cv2.threshold(blurred, 10 , 255, cv2.THRESH_BINARY)
  mask = mask/255

  # Extract the NCC maps
  red_NCC, green_NCC, blue_NCC = image_template_matching(img, template_images, mask)

  # Extract superpixel map
  if bright_on:
    brightness_map = extract_BR_map(src, mask)
  else:
    brightness_map = zeros

  # Combining localization maps 
  combined_map = red_NCC * r_coeff + green_NCC*g_coeff + blue_NCC*b_coeff + brightness_map*br_coeff
  all_maps = [combined_map, red_NCC, green_NCC, blue_NCC, brightness_map]
  
  # Extracting maximum value of NCC
  y, x = list(zip(*np.where(combined_map==np.max(combined_map))))[0]
  disk_center = (x, y)

  # Return all maps for testing the algorithm only
  if test_on:
    return disk_center, all_maps 
  elif not test_on:
    return disk_center

In [ ]:
def ekstrakROI(centroid, s, img):
  '''
    crop the region of interest based on OD's estimated center 

    Parameters
    ----------
    centeroid : tuple of int (x, y)
      estimated center of OD
    s: float
      size of ROI compared to the whole image
      value between 0 - 1
    img: array of int
      source image

    Returns
    ----------
    ROI: array of int
      region of interest
    koordinat: tupple (y0, y1, x0, x1)
      coordinates value of ROI trim
  '''

  h, w = img.shape[:2]
 
  y0, y1, x0, x1 = rectfromcenter(centroid, s, h, w)

  #cropping ROI from source image
  ROI = img[y0:y1, x0:x1]
  koordinat = (y0, y1, x0, x1)

  return ROI, koordinat

In [ ]:
def rectfromcenter(center, s, h, w):
  '''
    calculate x and y value of a rectangle based of a coordinate

    Parameters
    ----------
    center : tuple of int (x, y)
      coordinate value of rectangle's center
    s: float
      size of ROI compared to the whole image
      value between 0 - 1
    h: int
      image height
    w: int
      image width

    Returns
    ----------
    y0: int
      the bottommost point of the rectangle
    y1: int
      the topmost point of the rectangle
    x0: int
      the leftmost point of the rectangle
    x1: int
      the righttmost point of the rectangle
  '''
  x, y = center
  x0 = math.floor(x - 0.5*s)
  x1 = math.floor(x + 0.5*s)
  y0 = math.floor(y - 0.5*s)
  y1 = math.floor(y + 0.5*s)

  if (x0 < 0):
    x1 = x1 + (-x0)
    x0 = 0

  elif (x1 > w-1):
    x0 = x0 - (x1-(w-1))
    x1 = w-1

  if (y0 < 0):
    y1 = y1 + (-y0)
    y0 = 0

  elif (y1 > h-1):
    y0 = y0 - (y1-(h-1))
    y1 = (h-1)

  return y0, y1, x0, x1

### Segmentation Functions

In [ ]:
def ellips_fittingOD(mask):
  '''
    Fitting the predicted area of OD into an an ellipse with circle
    hough transform

    Parameters
    ----------
    mask: array of int
      OD prediction mask

    Returns
    ----------
    ellips: array of int
      fitted OD mask
    center: tuple of int (x, y)
      coordinate values of ellipse
    d: float
      major axis of the ellipse
    angle: float
      angle of major axis
  ''' 
  mask = mask.squeeze()

  cnts, _= cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
  elips = np.zeros((mask.shape[0], mask.shape[1]),np.uint8)
  if len(cnts)>0:
  # Ellipse fitting
    ellipse = cv2.fitEllipse(cnts[0])
    center, d, angle = ellipse
    elips = cv2.ellipse(elips, ellipse, 255, cv2.FILLED)
  else:
    elips = mask
    center = (0, 0)
    d = (0, 0)
    angle = 0
  
  ellips = np.expand_dims(elips, axis=-1)
  ellips = np.array(elips)
  ellips = elips.squeeze()
  return ellips, center, d, angle

In [ ]:
def check_hier(mask):
  '''
    Calculate how many blobs with more than five
    data point detected

    Parameters
    ----------
    mask: array of int
      OC prediction mask

    Returns
    ----------
    hier: int
      number of blobs found
  ''' 
  mask = mask.squeeze()
  cnts, _= cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  hier = 0
  for cnt in cnts:
    if len(cnt) > 5:
      hier += 1
  return hier

In [ ]:
def ellips_fittingOC(mask, hier):
  '''
    Fitting the predicted area of OC into an an ellipse with circle
    hough transform

    Parameters
    ----------
    mask: array of int
      OC prediction mask

    Returns
    ----------
    ellipsfitted: array of int
      fitted OC mask
  ''' 
  
  mask = mask.squeeze()
  # Select largest contour
  cnts, _= cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
  elips = np.zeros((mask.shape[0], mask.shape[1]),np.uint8)
  if len(cnts)>0:
  # Ellipse fitting
    if hier > (len(cnts)-1):
      hier = len(cnts) - 1
    ellipse = cv2.fitEllipse(cnts[hier])
    elips = cv2.ellipse(elips, ellipse, 255, cv2.FILLED)
  else:
    elips = mask

  ellips = np.expand_dims(elips, axis=-1)
  ellips = np.array(elips)
  ellips = elips.squeeze()
  return ellips

In [ ]:
def ellipse_fitting(OD, OC):
  '''
    Driver function for ellipse fitting

    Parameters
    ----------
    OD_bin: array of int
      OD prediction mask
    OC_bin: array of int
      OC prediction mask

    Returns
    ----------
    OD_fit: array of int
      fitted OD mask
    OC_fit: array of int
      fitted OC mask
  '''
  OD_fitted = []
  OC_fitted = []
  for i in range(len(OD)):
    OD_bin = OD[i]
    OC_bin = OC[i]
    OD_fit, _, _, _ = ellips_fittingOD(OD_bin)
    hier = 0
    hier_max = check_hier(OC_bin)
    if hier_max > 0:
      OC_fit = ellips_fittingOC(OC_bin, 0)
      check = np.sum(OC_fit[OD_fit == 255])/ np.sum(OC_fit[OC_fit == 255])
      hier += 1

      while (check < 0.3) and hier < (hier_max):
        OC_fit = ellips_fittingOC(OC_bin, hier)
        check = np.sum(OC_fit[OD_fit == 255])/ np.sum(OC_fit[OC_fit == 255])
        hier += 1
        if hier > hier_max:
          break
    else:
      OC_fit = OC_bin.squeeze()*255
    
    OD_fitted.append(OD_fit)
    OC_fitted.append(OC_fit)
  
  OD_fitted = np.array(OD_fitted)
  OC_fitted = np.array(OC_fitted)

  return OD_fitted, OC_fitted

### Feature Extractions Functions



In [ ]:
def CDRcalc(OD_feat, OC_feat):
  '''
    Calculate cup to disc ratio based on extracted features

    Parameters
    ----------
    OD_feat: tuple of float
      features of optic disc
      (height, length, area)
    OC_feat: tuple of float
      features of optic cup
      (height, length, area)

    Returns
    ----------
    VCDR: float
      Vertical cup-to-disc ratio
    HCDR: float
      Horizontal cup-to-disc ratio
    ACDR: float
      Area cup-to-disc ratio
  '''
  VCDR = float(OC_feat[0]) / float(OD_feat[0]) # vertical CDR
  HCDR = float(OC_feat[1]) / float(OD_feat[1]) # Horizontal CDR
  ACDR = float(OC_feat[2]) / float(OD_feat[2]) # Area CDR

  return VCDR, HCDR, ACDR

In [ ]:
def getFeature(mask):
  '''
    Extract features such as length, height, and area from
    OD or OC mask

    Parameters
    ----------
    mask: array of int
      Segmentation mask of OD or OC

    Returns
    ----------
    ver: float
      Height of mask
    hor: float
      Length of mask
    area: int
      Area of mask
  '''
  cont, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  cont = cv2.approxPolyDP(cont[0], 3, True)
  _, _, hor, ver = cv2.boundingRect(cont)
  area = np.sum(mask == 255)

  return ver, hor, area

In [ ]:
def getCenter(mask):
  '''
    Compute center of mask

    Parameters
    ----------
    mask: array of int
      Segmentation mask of OD or OC

    Returns
    ----------
    center: tuple of int
      Mask's center
      (x, y)
  '''
  M = cv2.moments(mask)
  center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

  return center

In [ ]:
def getODAxes(mask):
  '''
    Driver function for feature extraction of OD
    Compute its center, features, and angle of axis

    Parameters
    ----------
    mask: array of int
      Segmentation mask of OD

    Returns
    ----------
    center: tuple of int
      Mask's center
      (x, y)
    feat: tuple of float
      features of OD
      (height, width, area)
    angle: float
      angle of tilt
  '''
  c_OD, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
  ellipse = cv2.fitEllipse(c_OD[0])
  center, (hor, ver), angle = ellipse
  area = np.sum(mask == 255)
  
  if (angle > 45) and (angle <= 135):
    angle -= 90
    temp = hor
    hor = ver
    ver = temp

  elif (angle > 135) and (angle <= 180):
    angle -= 180

  return center, (ver, hor, area), angle

In [ ]:
def getOCAxes(mask, angle):
  '''
    Driver function for feature extraction of OC
    Compute its center, features, and angle of axis

    Parameters
    ----------
    mask: array of int
      Segmentation mask of OC
    angle: float
      tilt of OD

    Returns
    ----------
    center: tuple of int
      Mask's center
      (x, y)
    feat: tuple of float
      features of OC
      (height, width, area)
    angle: float
      angle of tilt
  '''
  center = getCenter(mask)
  rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
  mask = cv2.warpAffine(mask, rot_mat, mask.shape[1::-1], flags=cv2.INTER_LINEAR)
  feat = getFeature(mask)

  return center, feat, angle

### Driver Functions

In [ ]:
def uploadImage():
  uploaded = files.upload()
  src_image = []
  filenames = list(uploaded.keys())
  for image_path in filenames:
    img = cv2.imread(image_path, 1) # reading image retina
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Konversi warna BGR ke RGB
    src_image.append(img)
  return src_image, filenames

In [ ]:
def localization(src_image, image_templates):
  r_coeff, g_coeff,b_coeff, br_coeff = 1, 0.2, 0, 0.8
  used_channels = 'rg'
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  
  loc_imgs = []
  for img in src_image:
    disc_center, all_maps = OD_Localization(img, image_templates,
                                            used_channels=used_channels, 
                                            bright_on=True, 
                                            test_on=True, 
                                            r_coeff=r_coeff, 
                                            g_coeff=g_coeff, 
                                            b_coeff=b_coeff,
                                            br_coeff=br_coeff)
    img = clahe.apply(img[:,:,1])
    shape = (img.shape[1], img.shape[0])
    img,_ = ekstrakROI(disc_center, (0.3*shape[0]), img)
    img = resize(img, (550, 550, 1), mode = 'constant', preserve_range = True)
    loc_imgs.append(img)
  
  loc_imgs = np.array(loc_imgs)
  return loc_imgs

In [ ]:
def segmentation(img):
  img = resize(img, (len(img), 256, 256, 1),mode = 'constant',
               preserve_range = True)/255.0
  img = np.array(img)
  # Predict 
  OD = model_OD.predict(img, verbose=0)
  OC = model_OC.predict(img, verbose=0)
  # binarize
  OD_bin = (OD > 0.5).astype(np.uint8)
  OC_bin = (OC > 0.5).astype(np.uint8)
  
  return ellipse_fitting(OD_bin, OC_bin)

In [ ]:
def featureExtraction(OD, OC):
  CDR = []
  for i in range(len(OD)):
    _, featOD, angle = getODAxes(OD[i])
    _, featOC, _ = getOCAxes(OC[i], angle)
    CDR.append(CDRcalc(featOD, featOC))
  
  CDR = np.array(CDR)
  return CDR

In [ ]:
def glaucomaClassification(CDR, filenames):
  preds = model2a_axes.predict([(i[0], i[2]) for i in CDR])
  classes = ['Normal', 'Glaucoma']
  preds = [classes[x] for x in preds]
  print('%-3s | %-18s | %-10s' % ("No.", "File Name", "Prediction"))
  print('%-3s | %-18s | %-10s' % ("---", "------------------", "----------"))
  for i in range(len(preds)):
    print('%-3i | %-18s | %-10s' % (i + 1, filenames[i], preds[i]))

In [ ]:
# Display analysis functions
'''
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
def showImg(img):
  if img.shape[2] == 1:
    plt.imshow(img, cmap='gray')
  else:
    plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  
  plt.show()

def showSegmentation(ROI, OD, OC):
  plt.imshow(ROI, cmap='gray')
  plt.xticks([])
  plt.yticks([])
  plt.contour(OD, colors='royalblue')
  plt.contour(OC, colors='darkorange')
  plt.show()

def showFeatures(ROI, OD, OC):
  centerOD, featOD, angle = getODAxes(OD)
  centerOC, featOC, _ = getOCAxes(OC, angle)
  V, H, A = CDRcalc(featOD, featOC)

  boxOD = cv2.boxPoints((centerOD, tuple((featOD[1], featOD[0])), angle))
  boxOC = cv2.boxPoints((centerOC, tuple((featOC[1], featOC[0])), angle))

  plt.imshow(ROI, cmap='gray')
  plt.text(10, 80,
           'VCDR: {:.3f}\nHCDR: {:.3f}\nACDR: {:.3f}'.format(V, H, A),
           size='small', color='w')
  plt.contour(OD, colors='royalblue')
  plt.contour(OC, colors='darkorange')
  plt.gca().add_patch(Rectangle((boxOD[1]), featOD[1], featOD[0],
                                angle=angle, edgecolor='red',
                                facecolor='none', lw=2))
  plt.gca().add_patch(Rectangle((boxOC[1]), featOC[1], featOC[0],
                                angle=angle, edgecolor='red',
                                facecolor='none', lw=2))
  plt.plot(centerOC[0], centerOC[1], 'o', color='blue')
  plt.grid(False)
'''

"\nfrom matplotlib import pyplot as plt\nfrom matplotlib.patches import Rectangle\ndef showImg(img):\n  if img.shape[2] == 1:\n    plt.imshow(img, cmap='gray')\n  else:\n    plt.imshow(img)\n  plt.xticks([])\n  plt.yticks([])\n  \n  plt.show()\n\ndef showSegmentation(ROI, OD, OC):\n  plt.imshow(ROI, cmap='gray')\n  plt.xticks([])\n  plt.yticks([])\n  plt.contour(OD, colors='royalblue')\n  plt.contour(OC, colors='darkorange')\n  plt.show()\n\ndef showFeatures(ROI, OD, OC):\n  centerOD, featOD, angle = getODAxes(OD)\n  centerOC, featOC, _ = getOCAxes(OC, angle)\n  V, H, A = CDRcalc(featOD, featOC)\n\n  boxOD = cv2.boxPoints((centerOD, tuple((featOD[1], featOD[0])), angle))\n  boxOC = cv2.boxPoints((centerOC, tuple((featOC[1], featOC[0])), angle))\n\n  plt.imshow(ROI, cmap='gray')\n  plt.text(10, 80,\n           'VCDR: {:.3f}\nHCDR: {:.3f}\nACDR: {:.3f}'.format(V, H, A),\n           size='small', color='w')\n  plt.contour(OD, colors='royalblue')\n  plt.contour(OC, colors='darkorange')\n  

# Run Glaucoma Detection

In [ ]:
# Upload fundus image
src_img, filenames = uploadImage()

In [ ]:
# Run localization
ROI_img = localization(src_img, image_templates)

In [ ]:
# Run Segmentation
OD, OC = segmentation(ROI_img)

In [ ]:
# Run feature extraction
CDR = featureExtraction(OD, OC)

In [ ]:
# Run glaucoma classification
glaucomaClassification(CDR, filenames)

No. | File Name          | Prediction
--- | ------------------ | ----------
1   | drishtiGS_010.png  | Glaucoma  
2   | drishtiGS_092.png  | Normal    
3   | n0294.jpg          | Normal    
4   | n0317.jpg          | Normal    
